In [4]:
! pip3 install yfinance --upgrade --no-cache-dir
! pip3 install yahoo_fin
! pip3 install multiprocess

! pip3 install pyarrow

Requirement already up-to-date: yfinance in /Users/amberyeh/opt/anaconda3/lib/python3.8/site-packages (0.1.59)
     |████████████████████████████████| 15.7 MB 5.0 MB/s eta 0:00:01


In [5]:
import json
import pandas as pd
import numpy as np
from yahoo_fin import stock_info
import yahoo_fin
from multiprocessing import Pool
import multiprocess as mp
import datetime
from collections import Counter


In [6]:
# we have transactions data
all_transactions_parquet_file_name = '../collected_data/house_stock_watcher_data_all_transactions.parquet'
all_transactions = pd.read_parquet(all_transactions_parquet_file_name)

In [7]:
end_date = datetime.datetime.now().date()
start_date = end_date - datetime.timedelta(days=365 * 2)

# def get_historical_data(ticker, start_date=start_date, end_date=end_date):
# #     try:
#         return stock_info.get_data(
#             ticker, start_date=start_date, end_date=end_date, index_as_date = False, interval="1d")
# #     except:
# #         # really don't do this, it's hacky but whatever
# #         return ('ERROR', ticker)

# # now we have some way of getting stock info from multiple tickers

from get_historical_data import get_historical_data

def get_historical_data_batch(tickers, start_date, end_date):
    historical_data = []
    with Pool(32) as p:
        historical_data = p.map(get_historical_data, tickers)
    return historical_data


In [8]:
%%time
# stock_info.get_data('amzn')
get_historical_data_batch(['amzn'],start_date, end_date)

CPU times: user 70.8 ms, sys: 161 ms, total: 232 ms
Wall time: 1.59 s


[          date         open         high          low        close  \
 0   2019-06-03  1760.010010  1766.290039  1672.000000  1692.689941   
 1   2019-06-04  1699.239990  1730.819946  1680.890015  1729.560059   
 2   2019-06-05  1749.599976  1752.000000  1715.250000  1738.500000   
 3   2019-06-06  1737.709961  1760.000000  1726.130005  1754.359985   
 4   2019-06-07  1763.699951  1806.250000  1759.489990  1804.030029   
 ..         ...          ...          ...          ...          ...   
 498 2021-05-24  3215.500000  3257.949951  3210.500000  3244.989990   
 499 2021-05-25  3266.669922  3279.820068  3213.760010  3259.050049   
 500 2021-05-26  3274.590088  3295.729980  3258.510010  3265.159912   
 501 2021-05-27  3256.000000  3260.360107  3230.040039  3230.110107   
 502 2021-05-28  3242.000000  3247.989990  3219.699951  3223.070068   
 
         adjclose   volume ticker  
 0    1692.689941  9098700   AMZN  
 1    1729.560059  5679100   AMZN  
 2    1738.500000  4239800   AMZN  
 3

In [9]:
%%time

# a couple of hard-coded company->ticker values
company_to_ticker = {
    'Broadcom Inc.': 'AVGO',
    'AD': 'ADM',
    'ADDDYY': 'ADDYY',
    'LBDAV': 'LBRDA',
    'LCRX': 'LRCX',
    'LEN.B': 'LEN-B',
    'LTD': 'LB',
    'MRCK': 'MRK',
    'MSBHY': 'MSBHF',
    'NGLS': 'NGL',
    'ORLT': 'ORLY',
    'POL': 'AVNT',
    'RBS': 'NWG',
    'RDS.A': 'RDS-A',
    'RDS.B': 'RDS-B',
    'RF$A': 'RF-A',
    'SERV': 'TMX',
    'SVCBY': 'SVCBF',
    'TDDC': 'TDC',
    'URGO': 'UGRO',
    'WFC$V': 'WFC',
    'WYND': 'TNL',
    'XMO': 'XOM',
    'ZOOM': 'ZM'
}
all_transactions['cleaned_ticker'] = all_transactions['ticker'].apply(lambda ticker: company_to_ticker.get(ticker, ticker))
# replace '.' w/ '-' to make things compatible
all_transactions['cleaned_ticker'] = all_transactions['cleaned_ticker']

# get the set of tickers
tickers = sorted(set(all_transactions['cleaned_ticker']))
# '--' is the null ticker, remove it
if '--' in tickers:
    tickers.remove('--')

historical_data = get_historical_data_batch(tickers, start_date, end_date)


CPU times: user 1.18 s, sys: 336 ms, total: 1.51 s
Wall time: 50.1 s


In [10]:
Counter([isinstance(el, tuple) and 'ERROR' == el[0] for el in historical_data])

Counter({False: 1659, True: 103})

In [11]:
lemon_tickers = [el[1] for el in historical_data if isinstance(el, tuple)]

In [19]:
# AGN: company purchased by Abbvie (ABBV) in May 2020
# AMTD: company purchased by Charles Schwab (SCHW)
lemon_tickers[0:]


['AGN',
 'AMTD',
 'AOBC',
 'APC',
 'APCD',
 'APPL',
 'AZSEY',
 'BAC$K',
 'BAMXY',
 'BBCBX',
 'BGG',
 'BOA',
 'BPHSPX',
 'BRK.A',
 'BRK.B',
 'BXS$A',
 'CBS',
 'CCC',
 'CCXX',
 'CELG',
 'CELO',
 'CHL',
 'CHV',
 'CTL',
 'CTRCF',
 'CWEN.A',
 'DCMYY',
 'DEACU',
 'DESY',
 'DNKN',
 'DWDP',
 'EBJ',
 'EQUIX',
 'ETFC',
 'ETP',
 'EVGB',
 'FFHRX',
 'FII',
 'FMCMF',
 'FNRVGX',
 'FXM',
 'GEAGY',
 'GLIBA',
 'HCN',
 'HDS',
 'HS',
 'HTZ',
 'HZD',
 'IDXXX',
 'INTL',
 'JPM$G',
 'LAM',
 'LBDAV',
 'LCRX',
 'LEN.B',
 'LLL',
 'LPT',
 'LTD',
 'MINI',
 'MNTA',
 'MRCK',
 'MSBHY',
 'MYL',
 'MZOR',
 'NBL',
 'NGLS',
 'NLG',
 'ORLT',
 'PEGI',
 'POL',
 'QCHR',
 'RBS',
 'RDS.A',
 'RDS.B',
 'RF$A',
 'RTN',
 'SERV',
 'SKVKY',
 'SPDR',
 'SPN',
 'SPYS',
 'SVCBY',
 'SZEVY',
 'TDDC',
 'TF',
 'TIF',
 'TMK',
 'TMUSR',
 'URGO',
 'UTX',
 'VIAB',
 'VMN',
 'VRTU',
 'VSLR',
 'WAIR',
 'WCG',
 'WFC$V',
 'WLTL',
 'WMGI',
 'WPX',
 'WYND',
 'XMO',
 'ZOOM']

In [ ]:
#LBDAV -> LBRDA
#LCRX -> LRCX
#LEN.B -> LEN-B
#LLL : only in CNSX
#LPT (seems okay, not on Yahoo Finance though)
#LTD -> LB
#MINI : combined with Willscot (WSC)
#MNTA : sold to Johnson & Johnson
#MRCK -> MRK
#MSBHY -> MSBHF
#MYL : merged with Pfizer's Upjohn to form Viatris (VTRS)
#MZOR : acquired by Medtronic (MDT)
#NBL : acquired by Chevron
#NGLS -> NGL
#NLG : Nam Long Investment Corp (not in USD)
#ORLT -> ORLY
#PEGI : takeover by Canada Pension Plan Investment Board (CPPIB), now a private company
#POL -> AVNT (aka Avient Corp)
#QCHR : only in NASDAQ
#RBS -> NWG (formerly Royal Bank of Scotland, now NatWest)
#RDS.A -> RDS-A
#RDS.B -> RDS-B (based in UK, no tax-witholding penalty)
#RF$A -> RF-A
#RTN : only in JSE (Africa)
#SERV -> TMX (changed name to Terminix)
#SKVKY -> SEB-A.ST (currency in Swedish money)
#SPDR : is an ETF
#SPN : only in CVE (Canada)
#SPYS : SPY? is an ETF
#SVCBY -> SVCBF
#SZEVY : In France (SEV.PA)
#TDDC -> TDC
#TF : In Canada (TF.TO)
#TIF : purchased by Louis Vitton
#TMK : only in ASX (Australia)
#TMUSR : right to buy 0.05 shares of TMUS (T-mobile) at a discounted price
#URGO -> UGRO
#UTX : merged with Raytheon (RTX), plus spinoffs?
#VIAB : merged with CBS, shares converted to CBS
#VMN : could be VNM (Vietnam), or an ETF?
#VRTU : acquired and privatized by Baring Private Equity Asia (BPEA)
#VSLR : acquired by Sunrun Inc (RUN)
#WAIR : acquired and privatized by Platinum Equity affiliate ()
#WCG : only in ASX (Australia)
#WFC$V -> WFC
#WLTL : only in NASDAQ
#WMGI : only in NASDAQ
#WPX : merged with Devon Energy, 0.5165 shares of DVN for each WPX
#WYND -> TNL (aka Travel + Leisure Co.)
#XMO -> XOM
#ZOOM -> ZM

In [36]:
lemon_tickers[0]

index = 9
print(lemon_tickers[index])


all_transactions[all_transactions['cleaned_ticker'] == lemon_tickers[index]]

BBCBX


,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd,cleaned_ticker
7974,2021,04/01/2021,2021-03-25,None,BBCBX,Bridge Builder Core Plus Bond @ 10.29,purchase,"$1,001 - $15,000",Hon. Harold Dallas Rogers,KY05,https://disclosures-clerk.house.gov/public_dis...,False,BBCBX


In [37]:
stock_info.get_data('bbcbx')

AssertionError: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}